In [ ]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
import openai

# OpenAI API 키 설정
openai.api_key = "YOUR_API_KEY"  # 실제 API 키로 대체

# 모델 및 클라이언트 초기화
model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient()

# 기존 컬렉션 삭제 후 생성
if "crawled_data" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="crawled_data")
collection = client.create_collection(name="crawled_data")

# 현재 디렉토리 내 모든 .txt 파일 찾기
# 밑에 경로 수정
txt_files = [file for file in os.listdir('/content') if file.endswith('.txt')]

# 라벨링 함수
def label_document(content):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 문서를 분석하여 적절한 주제를 할당하는 어시스턴트입니다. 가능한 주제는 '주거', '취업', '문화활동', '금융'입니다. 각 문서를 분석하여 가장 관련이 높은 주제를 선택하세요."},
                {"role": "user", "content": f"다음 문서의 주제를 분석하고 적합한 주제를 반환하세요:\n\n{content}"}
            ]
        )
        label = response.choices[0].message.content.strip()
        return label
    except Exception as e:
        print(f"라벨링 실패: {e}")
        return "기타"  # 실패 시 기본 라벨

# 파일 내용을 읽어 데이터베이스에 추가
for idx, txt_file in enumerate(txt_files):
    file_path = os.path.join('/content', txt_file)
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read().strip()  # 파일 내용 읽기
        if content:  # 내용이 있는 경우만 처리
            # 라벨링
            label = label_document(content)
            print(f"파일 '{txt_file}'에 대한 라벨: {label}")
            
            # 임베딩 및 데이터 저장
            embedding = model.encode([content])[0]
            collection.add(
                documents=[content],
                embeddings=[embedding],
                metadatas=[{"label": label}],
                ids=[f"doc_{idx+1}"]
            )
            print(f"파일 '{txt_file}'의 내용을 저장했습니다.")

print("모든 파일의 데이터 저장이 완료되었습니다.")